In [4]:
import pandas as pd

df = pd.read_csv('./synthetic-browsing-history/Israel/synthetic-browsing-history-IL_0.csv')

In [6]:
df.head()

,synthetic_time,synthetic_url,original_scope,original_content,original_country_rank,original_world_rank,original_lang,original_path_bin
0,2024-11-01 09:32:55.373550,https://www.tomsguide.com/computing/vpns/can-y...,Global,Business,-1,10000,eng,50
1,2024-11-01 10:18:53.105916,https://dev.to/harshdand/react-performance-opt...,Global,Society,-1,10000,eng,-1
2,2024-11-01 10:19:23.803112,https://www.digitaltrends.com/home-theater/5-t...,Global,Society,500,1000,eng,-1
3,2024-11-01 10:22:03.663594,https://www.wondershare.com/ai-solutions/busin...,Global,Society,-1,10000,eng,50
4,2024-11-01 10:22:24.705302,https://www.digitaltrends.com/gaming/destiny-2...,Global,Society,500,1000,eng,-1


In [9]:
df['original_content'].value_counts()

original_content
Society     357
Shopping    136
News         69
Business     37
Generic      20
Name: count, dtype: int64

In [20]:
import pandas as pd

ads_df = pd.read_csv('./synthetic-browsing-history/ads_df.csv')
products_df = pd.read_csv('./synthetic-browsing-history/products_df.csv')
events_df = pd.read_csv('./synthetic-browsing-history/events_df.csv')

In [21]:
events_df.head()

,events_id,User_id,ad_id,Interaction_score,device_type,geo_location,Product_id,product_name,product_category,conversion_value,timestamp
0,ba44d39c-19a6-4906-a336-a0aec8b74168,84,23.0,1,desktop,Tammyview,4,Herman Miller Chair,office,0.0,2024-06-15T03:26:03.892436
1,260829e4-c2b4-43ae-9bd6-83c868ee1b3c,66,37.0,0,mobile,South John,32,Canon EOS M50,electronics,0.0,2024-08-27T03:26:03.892573
2,c642fa03-5d5b-4e2b-a604-17b13dc6e73a,79,NaN,2,tablet,South Michael,39,Fossil Smartwatch,accessories,0.0,2024-04-30T03:26:03.892675
3,25f234d7-e34e-4b93-8c2f-da41b51860cd,60,4.0,1,desktop,Port Nathanview,22,Schwinn Mountain Bike,outdoor,0.0,2024-07-30T03:26:03.892778
4,af4684f3-5bb8-41c2-af43-c4824d89d526,24,NaN,4,mobile,East Nichole,3,Hydro Flask,beverages,45.0,2024-12-10T03:26:03.892873


In [15]:
products_df.head()

,id,name,category,price,company
0,1,Air Force 1s,fashion,120.00,TrendWear
1,2,MacBook Pro,electronics,1999.99,TechCo
2,3,Hydro Flask,beverages,45.00,GourmetWorld
3,4,Herman Miller Chair,office,1400.00,OfficePro
4,5,Dell XPS 13,electronics,1099.99,TechCo


In [17]:
ads_df.head()

,Ad_id,AdText,AdCategory
0,1,Discover the amazing Air Force 1s from TrendWe...,fashion
1,2,Discover the amazing MacBook Pro from TechCo! ...,electronics
2,3,Discover the amazing Hydro Flask from GourmetW...,beverages
3,4,Discover the amazing Herman Miller Chair from ...,office
4,5,Discover the amazing Dell XPS 13 from TechCo! ...,electronics


In [40]:
def recommend_ads(user_id, device_type):
    # Filter user-specific events
    user_events = events_df[events_df["User_id"] == user_id]

    # Extract preferences
    preferred_categories = user_events["product_category"].value_counts().index.tolist()
    preferred_companies = user_events["product_name"].value_counts().index.tolist()
    preferred_device = user_events["device_type"].value_counts().idxmax()

    # Ads already seen by the user
    seen_ads = user_events["ad_id"].dropna().unique()

    # Filter ads not yet seen
    ads_candidates = ads_df[
        ~ads_df["Ad_id"].isin(seen_ads) &  # Exclude seen ads
        ads_df["AdCategory"].isin(preferred_categories)  # Match preferred categories
    ].copy()  # Ensure a deep copy to avoid warnings

    # Score ads based on relevance
    ads_candidates.loc[:, "score"] = ads_candidates["AdCategory"].apply(
        lambda cat: preferred_categories.index(cat) + 1 if cat in preferred_categories else 0
    )
    ads_candidates = ads_candidates.sort_values("score", ascending=False)

    ads_candidates.loc[:, "company_match"] = ads_candidates["AdText"].apply(
        lambda text: any(company in text for company in preferred_companies)
    )
    ads_candidates.loc[:, "score"] += ads_candidates["company_match"] * 1.2

    ads_candidates.loc[:, "device_match"] = ads_candidates["Ad_id"].apply(
        lambda ad: device_type == preferred_device
    )
    ads_candidates.loc[:, "score"] += ads_candidates["device_match"] * 1.2

    ads_candidates = ads_candidates.sort_values("score", ascending=False)
    top_ads = ads_candidates.head(5)

    return top_ads["AdText"].tolist()


recommend_ads(1, 'mobile')

/var/folders/fs/sd_gznt92_1387bmvbkzdlb40000gn/T/ipykernel_97971/2725827281.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[5.2 4.  3.  3.  3.  4.2 3.  3.  3.  3.  4.2 3.  3.  3.  3.  3.2 3.2 3.2
 2.  2.2 2.2 1.  2.2 1.  2.2]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ads_candidates.loc[:, "score"] += ads_candidates["company_match"] * 1.2


['Discover the amazing Peloton Bike from FitLife! Perfect for all your fitness needs.',
 'Discover the amazing GoPro Hero 11 from TechCo! Perfect for all your electronics needs.',
 'Discover the amazing Google Nest Hub from TechCo! Perfect for all your electronics needs.',
 'Discover the amazing Fitbit Charge 5 from FitLife! Perfect for all your fitness needs.',
 'Discover the amazing Weber Charcoal Grill from HomeStyle! Perfect for all your home needs.']

In [41]:
len(events_df['User_id'].unique())

100

In [44]:
user_event_counts = events_df.groupby("User_id").size().reset_index(name="event_count")

In [46]:
mean_events = user_event_counts["event_count"].mean()
median_events = user_event_counts["event_count"].median()
std_events = user_event_counts["event_count"].std()

print(mean_events)
print(median_events)
print(std_events)

10.0
10.0
2.9267142951877907


In [47]:
events_df.head()

,events_id,User_id,ad_id,Interaction_score,device_type,geo_location,Product_id,product_name,product_category,conversion_value,timestamp
0,ba44d39c-19a6-4906-a336-a0aec8b74168,84,23.0,1,desktop,Tammyview,4,Herman Miller Chair,office,0.0,2024-06-15T03:26:03.892436
1,260829e4-c2b4-43ae-9bd6-83c868ee1b3c,66,37.0,0,mobile,South John,32,Canon EOS M50,electronics,0.0,2024-08-27T03:26:03.892573
2,c642fa03-5d5b-4e2b-a604-17b13dc6e73a,79,NaN,2,tablet,South Michael,39,Fossil Smartwatch,accessories,0.0,2024-04-30T03:26:03.892675
3,25f234d7-e34e-4b93-8c2f-da41b51860cd,60,4.0,1,desktop,Port Nathanview,22,Schwinn Mountain Bike,outdoor,0.0,2024-07-30T03:26:03.892778
4,af4684f3-5bb8-41c2-af43-c4824d89d526,24,NaN,4,mobile,East Nichole,3,Hydro Flask,beverages,45.0,2024-12-10T03:26:03.892873


In [48]:
products_df.head()

,id,name,category,price,company
0,1,Air Force 1s,fashion,120.00,TrendWear
1,2,MacBook Pro,electronics,1999.99,TechCo
2,3,Hydro Flask,beverages,45.00,GourmetWorld
3,4,Herman Miller Chair,office,1400.00,OfficePro
4,5,Dell XPS 13,electronics,1099.99,TechCo


In [50]:
ads_df.head()

,Ad_id,AdText,AdCategory
0,1,Discover the amazing Air Force 1s from TrendWe...,fashion
1,2,Discover the amazing MacBook Pro from TechCo! ...,electronics
2,3,Discover the amazing Hydro Flask from GourmetW...,beverages
3,4,Discover the amazing Herman Miller Chair from ...,office
4,5,Discover the amazing Dell XPS 13 from TechCo! ...,electronics


In [ ]:
import os
from datetime import datetime
from elevenlabs.client import ElevenLabs

def text_to_speech(text, user_id):
    """
    Convert text to audio and save it to a file.
    :param text: Text to convert to audio
    :param user_id: User ID for file naming
    :return: Path to the saved audio file
    """
    # Initialize ElevenLabs client
    client = ElevenLabs(api_key="YOUR_API_KEY")
    
    try:
        # Generate audio
        audio_stream = client.generate(text=text, voice='Brian', model="eleven_multilingual_v2")
    except Exception as e:
        print(f"Error generating audio: {e}")
        return None
    
    # Set output directory and file path
    output_dir = "generated_audio"
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
    file_path = os.path.join(output_dir, f'output_audio_{user_id}_{timestamp}.mp3')
    
    try:
        # Write audio data to the file
        with open(file_path, "wb") as audio_file:
            for chunk in audio_stream:
                audio_file.write(chunk)
        print(f"Audio saved to {file_path}")
        return file_path
    except Exception as e:
        print(f"Error saving audio file: {e}")
        return None

# Test the function with sample input
sample_text = "level up your coffee experience with the incredible keurig k-elite coffee maker! brew your favorite beverages effortlessly and enjoy café-quality drinks at home. don’t miss out—sip, savor, and elevate your mornings!"
sample_user_id = 73

text_to_speech(sample_text, sample_user_id)


Audio saved to generated_audio/output_audio_73_20241224234519.mp3


'generated_audio/output_audio_73_20241224234519.mp3'

In [1]:
!pip install ffmpeg
import ffmpeg
import random

def background_music(self, ad_file, volume = 0.4):

    background_music = random.choice(['music/BestPart.mp3', 'music/WildFlower.mp3'])
    output_file = f"../generated_audio/TESTWithBackground{ad_file.split('/')[-1]}"
    try:
        (
            ffmpeg
            .filter(
                [ffmpeg.input(ad_file), ffmpeg.input(background_music).filter('volume', volume).filter('atempo', tempo = 1.2)],
                'amix', inputs=2, duration='shortest'
            )
            .output(output_file, acodec='mp3')  # Output as MP3
            .run(overwrite_output=True)
        )
        print(f"Success. Submitted to {output_file}")
        return output_file
    except ffmpeg.Error as e:
        print(f"Error occurred: {e.stderr.decode()}")
        return None



zsh:1: command not found: pip


ModuleNotFoundError: No module named 'ffmpeg'